In [15]:
from craterdetection.matching.database import CraterDatabase
from craterdetection.matching.database import extract_robbins_dataset, load_craters
from craterdetection.matching.projective_invariants import crater_representation, matrix_adjugate

import numpy as np
import numpy.linalg as LA

In [5]:
%%time
lat, long, major, minor, psi, crater_id = extract_robbins_dataset(
    load_craters("../data/lunar_crater_database_robbins_2018.csv")
)

CPU times: user 3.76 s, sys: 570 ms, total: 4.33 s
Wall time: 4.36 s


#### Convert coordinates to 3D selenographic position

In [7]:
R_moon = 1737.1

p_Mi = R_moon*np.array([
    np.cos(lat)*np.cos(long),
    np.cos(lat)*np.sin(long),
    np.sin(lat)
]).T[:, :, None]

#### Define local ENU coordinate system

In [8]:
k = np.array([0, 0, 1])[:, None]
print(k.shape)

u_i = p_Mi / R_moon

e_i = np.cross(k.squeeze(), u_i.squeeze())[:, :, None]
e_i /= np.tile(LA.norm(e_i, ord=2, axis=(1,2)), (3, 1)).T[:, :, None]

n_i = np.cross(u_i.squeeze(), e_i.squeeze())[:, :, None]
n_i /= np.tile(LA.norm(n_i, ord=2, axis=(1,2)), (3, 1)).T[:, :, None]

T_EM = np.moveaxis(np.array([e_i, n_i, u_i]), 0, 1).squeeze()

(3, 1)


In [9]:
planes = np.concatenate([u_i, -R_moon*np.ones(p_Mi.shape[0])[:, None, None]], axis=1)
S = np.concatenate((np.identity(2), np.zeros((1, 2))), axis=0)
S = np.tile(S, (p_Mi.shape[0], 1, 1))

In [10]:
H_Mi = np.concatenate((T_EM @ S, p_Mi), axis=-1)

In [11]:
x_quadric = np.concatenate((H_Mi, np.tile(k.T, (p_Mi.shape[0], 1, 1))), axis=1)

In [16]:
C = crater_representation(0, 0, major, minor, psi)
C_star = matrix_adjugate(C)

In [24]:
Q_star = x_quadric @ C_star @ x_quadric.transpose(0, 2, 1)

In [25]:
Q_star

array([[[-9.46515333e+01,  2.92248503e+01,  1.64036627e+01,
         -1.64876266e-02],
        [ 2.92248503e+01,  2.76119549e+02,  7.38479660e+01,
         -1.76558521e-01],
        [ 1.64036627e+01,  7.38479660e+01, -4.63774024e+01,
         -6.41581112e-02],
        [-1.64876266e-02, -1.76558521e-01, -6.41581112e-02,
          1.08558107e-04]],

       [[ 1.94486983e+02,  3.22354313e+02, -1.48513690e+01,
         -2.40312263e-01],
        [ 3.22354313e+02,  3.92170934e+02, -1.50750684e+01,
         -2.92436042e-01],
        [-1.48513690e+01, -1.50750684e+01, -6.43260735e+01,
          1.28918725e-02],
        [-2.40312263e-01, -2.92436042e-01,  1.28918725e-02,
          2.18023236e-04]],

       [[-7.09377062e+01,  1.43996550e+01,  1.67396803e+01,
         -1.28074919e-02],
        [ 1.43996550e+01,  1.82746552e+02,  2.25344555e+02,
         -2.00938833e-01],
        [ 1.67396803e+01,  2.25344555e+02,  2.72949306e+02,
         -2.33124855e-01],
        [-1.28074919e-02, -2.00938833e-